## A jupyter notebook version file for the `main.py`


Set `autoreload` to execute the change in `.py` files.


In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import _pickle
import natsort
from pathlib import Path
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from __utils__ import image_processing
from __utils__ import label_processing
from __utils__ import labeling
from __utils__ import loso_preparing
from __utils__ import functions

In [19]:
dataset_dir = "D:/Databases/SAMM_longvideos"
# dataset_dir = "F:/HEH/Databases/SAMM_longvideos"
# dataset_dir = "/data/disk1/heh/databases/SAMM_longvideos"

images_loading = False
image_size = 128
load_cropped_images = False
expression_type = "mae"  # macro-expression spotting
# expression_type = "me"  # micro-expression spotting
debug_preds = True
labeling_function = "pseudo_labeling"
# labeling_function = "original_labeling"
model_names = {
    0: "SOFTNet",
    1: "SOFTNetCBAM",
    2: "ViT-B",
    3: "SL-ViT-B",
    4: "Swin-T",
    5: "Swin-S",
    6: "L-Swin-T",
    7: "S-Swin-T",
    8: "SL-Swin-T",
    9: "SL-Swin-S",
}
model_name = model_names[6]
batch_size = 48
epochs = 25
save_preds = False
preds_stem = (
    f"{expression_type}_"
    + model_name.lower().replace("-", "_")
    + f"_batch_size_{batch_size}"
    + f"_epochs_{epochs}"
    + f"_{labeling_function}"
    + f"_{image_size}"
    # + "_2"
)
preds_path = Path(dataset_dir, "preds", preds_stem).with_suffix(".pkl")
print(f"preds_path: {preds_path}")

preds_path: D:\Databases\SAMM_longvideos\preds\mae_l_swin_t_batch_size_48_epochs_25_pseudo_labeling_128.pkl


## Load Images


When debug the image processing, the videos_images is from cropped_rawpic, whereas the other variables are from rawpic.


In [20]:
videos_images, subjects, subjects_videos_code = image_processing.load_images(
    dataset_dir,
    images_loading=images_loading,
    image_size=image_size,
    load_cropped_images=load_cropped_images,
)

subject_video: 006_1
subject_video: 006_2
subject_video: 006_3
subject_video: 006_4
subject_video: 006_5
subject_video: 006_6
subject_video: 006_7
subject_video: 007_3
subject_video: 007_4
subject_video: 007_5
subject_video: 007_6
subject_video: 007_7
subject_video: 008_1
subject_video: 008_5
subject_video: 008_6
subject_video: 008_7
subject_video: 009_2
subject_video: 009_3
subject_video: 009_4
subject_video: 009_6
subject_video: 009_7
subject_video: 010_1
subject_video: 010_2
subject_video: 010_3
subject_video: 010_4
subject_video: 010_5
subject_video: 010_6
subject_video: 010_7
subject_video: 011_1
subject_video: 011_2
subject_video: 011_3
subject_video: 011_4
subject_video: 011_5
subject_video: 011_6
subject_video: 011_7
subject_video: 012_3
subject_video: 012_4
subject_video: 012_5
subject_video: 012_6
subject_video: 012_7
subject_video: 013_1
subject_video: 013_2
subject_video: 013_3
subject_video: 013_6
subject_video: 013_7
subject_video: 014_1
subject_video: 014_2
subject_video

In [21]:
print("subjects:", subjects)
print("subjects_videos_code:", subjects_videos_code)

subjects: ['006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '028', '030', '031', '032', '033', '034', '035', '036', '037']
subjects_videos_code: [['1', '2', '3', '4', '5', '6', '7'], ['3', '4', '5', '6', '7'], ['1', '5', '6', '7'], ['2', '3', '4', '6', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['3', '4', '5', '6', '7'], ['1', '2', '3', '6', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['1', '3', '5', '6', '7'], ['1', '2', '4', '5', '6', '7'], ['1', '2', '3', '4', '5', '6'], ['1', '2', '3', '4', '5', '6', '7'], ['1', '2', '3', '4', '5', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['3', '7'], ['2', '3', '4', '5', '6'], ['1', '4'], ['2', '3', '5'], ['3', '4', '5', '6'], ['1', '2', '3', '5', '6', '7'], ['4'], ['1', '2', '5'], ['3'], ['2', '3', '4', '5', '6'], ['1', '2', '3', '4', '5', '6', '7'], ['3', '6', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['2', '4',

## Load Excel 


In [22]:
Excel_data = label_processing.load_excel(dataset_dir)
Excel_data.head(5)

Data Columns: Index(['subject', 'Filename', 'video_code', 'onset', 'apex', 'offset',
       'Duration', 'expression_type', 'Action Units', 'Notes',
       'subject_video_code', 'subject_code'],
      dtype='object')


,subject,Filename,video_code,onset,apex,offset,Duration,expression_type,Action Units,Notes,subject_video_code,subject_code
0,006,006_1_1,1,566,648,743,178,Macro,4(B/C)+7B,NaN,006_1,006
1,006,006_1_2,1,3562,3588,3632,71,Micro - 1/2,4+7,NaN,006_1,006
2,006,006_1_3,1,1912,1948,1988,77,Micro - 1/2,4,While blinking,006_1,006
3,006,006_1_4,1,324,368,403,80,Micro - 1/2,4+7,NaN,006_1,006
4,006,006_1_5,1,3343,3388,3424,82,Micro - 1/2,4+7,NaN,006_1,006


## Load Ground Truth Labels


In [23]:
(
    clean_videos_images,
    clean_subjects_videos_code,
    clean_subjects,
    clean_subjects_videos_ground_truth_labels,
) = label_processing.load_ground_truth_labels(
    dataset_dir,
    expression_type,
    videos_images,
    subjects_videos_code,
    subjects,
    Excel_data,
)

required_videos_index:  [0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 95, 97, 98, 99, 100, 101, 103, 106, 107, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146]
len(clean_videos_images) = 130


In [24]:
print("len(clean_subjects): ", len(clean_subjects))
print("clean_subjects: ", clean_subjects)
print("len(clean_subjects_videos_code): ", len(clean_subjects_videos_code))
print("clean_subjects_videos_codes: ", clean_subjects_videos_code)
print(
    "len(clean_subjects_videos_ground_truth_labels): ",
    len(clean_subjects_videos_ground_truth_labels),
)
print(
    "clean_subjects_videos_ground_truth_labels: ",
    clean_subjects_videos_ground_truth_labels,
)

len(clean_subjects):  29
clean_subjects:  ['006' '007' '008' '009' '010' '011' '012' '013' '014' '015' '016' '017'
 '018' '019' '020' '021' '022' '023' '024' '025' '026' '028' '030' '032'
 '033' '034' '035' '036' '037']
len(clean_subjects_videos_code):  29
clean_subjects_videos_codes:  [['1', '2', '3', '4', '5', '6', '7'], ['5', '6', '7'], ['1', '5', '6', '7'], ['3', '4', '6', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['3', '4', '5', '6', '7'], ['1', '2', '3', '6', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['1', '3', '5', '6', '7'], ['1', '2', '4', '5', '6', '7'], ['1', '2', '3', '4', '5', '6'], ['2', '3', '4', '5', '6', '7'], ['1', '2', '3', '5', '7'], ['1', '2', '3', '4', '5', '6', '7'], ['3'], ['2', '3', '6'], ['4'], ['2', '3', '5'], ['3', '5'], ['2', '3', '7'], ['4'], ['1', '2', '5'], ['2', '3', '4', '5', '6'], ['1', '2', '3', '4', '5', '6', '7'], ['6', '7'], ['1', '2', '3', '4', '6', '7'], ['2', '4', '6', '7'], ['2', '3', '4', '5', '7']]
len(

## Calculate `k`


In [25]:
k = label_processing.calculate_k(clean_subjects_videos_ground_truth_labels)

k (Half of average length of expression) =  169


## Labeling


In [26]:
if labeling_function == "pseudo_labeling":
    labels = labeling.get_pseudo_labels(
        clean_videos_images, clean_subjects_videos_ground_truth_labels, k
    )
elif labeling_function == "original_labeling":
    labels = labeling.get_original_labels(
        clean_videos_images, clean_subjects_videos_ground_truth_labels, k
    )

Total frames: 438311


## Prepare for LOSO


In [27]:
y, groups = loso_preparing.prepare_for_loso(
    labels,
    clean_subjects,
    clean_videos_images,
    clean_subjects_videos_ground_truth_labels,
    k,
)

Frame Index for each subject:-

subject 006 ( group = 0): 0 -> 29116
subject 006 has 7 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  7

subject 007 ( group = 1): 29116 -> 42609
subject 007 has 3 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  10

subject 008 ( group = 2): 42609 -> 50824
subject 008 has 4 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  14

subject 009 ( group = 3): 50824 -> 55844
subject 009 has 4 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  18

subject 010 ( group = 4): 55844 -> 73934
subject 010 has 7 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  25

subject 011 ( group = 5): 73934 -> 116503
subject 011 has 7 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  32

subject 012 ( group = 6): 116503 -> 130313
subject 012 has 5 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  37

subject 013 ( group = 7): 130313 -> 151145
subject 013 has 5 clean 

## Training


In [28]:
if debug_preds is False:
    from __utils__.training_dev import train

    preds = train(
        dataset_dir,
        clean_subjects,
        image_size=image_size,
        y=y,
        expression_type=expression_type,
        model_name=model_name,
        train_or_not=True,
        batch_size=batch_size,
        epochs=epochs,
        preds_path=preds_path,
    )
else:
    with open(preds_path, "rb") as pkl_file:
        preds = _pickle.load(pkl_file)
        pkl_file.close()

In [29]:
if save_preds is True:
    with open(preds_path, "wb") as pkl_file:
        _pickle.dump(preds, pkl_file)
        pkl_file.close()

## Spotting and Evaluation


In [30]:
metric_fn, result_dict = functions.spot_and_evaluate(
    preds,
    clean_subjects_videos_ground_truth_labels,
    clean_videos_images,
    clean_subjects,
    clean_subjects_videos_code,
    k,
    p=0.60,
    show_plot_or_not=False,
)

Split 1/29 is in process.
0 video(s) have been processed.
The current video be processed: subject 006, video 1
The current video be processed: subject 006, video 2
The current video be processed: subject 006, video 3
The current video be processed: subject 006, video 4
The current video be processed: subject 006, video 5
The current video be processed: subject 006, video 6
The current video be processed: subject 006, video 7

True Positive: 4, False Posive: 12, False Negative: 19
Precision = 0.2500000000000002, Recall =0.1739130434782611, F1-Score = 0.2051282051282056
Split 1/29 is processed.

Split 2/29 is in process.
7 video(s) have been processed.
The current video be processed: subject 007, video 5
The current video be processed: subject 007, video 6
The current video be processed: subject 007, video 7

True Positive: 5, False Posive: 15, False Negative: 23
Precision = 0.2500000000000002, Recall =0.1785714285714288, F1-Score = 0.2083333333333338
Split 2/29 is processed.

Split 3/29

## Final Evaluation


In [31]:
functions.final_evaluate(metric_fn, result_dict)

True Positive: 60, False Posive: 226, False Negative: 271
Final Precision = 0.20979020979021,
Final Recall =0.1812688821752268,
Final F1-Score = 0.19448946515397125

Highest Precision = 0.33333333333333354,
Highest Recall =0.24242424242424265,
Highest F1-Score = 0.2807017543859654


| Parameters | Value
| --- | ---
| model | 3D-CNN
| epochs | | 
| batch_size | 
| learning_rate | 
| True Positive | 
| False Positive | 
| False Negative | 
| Precision | 
| Recall |  
| F1-Score | 0.1595 


| Parameters | Value | Value
| --- | --- | ---
| model | SL-Swin-T | SL-Swin-T
| epochs | 25 (9264 m) | 25 (9264 m)
| batch_size | 48 | 48
| p | 0.59 | 0.60
| True Positive | 54 | 52
| False Positive | 240 | 238
| False Negative | 277 | 279
| Precision | 0.1836 | 0.1793
| Recall | 0.1631 | 0.1571
| F1-Score | 0.1728 | 0.1675


| Parameters | Value | Value
| --- | --- | ---
| model | S-Swin-T | L-Swin-T
| epochs | 25 ( m) | 25 (14027 m)
| batch_size | 48 | 48
| p | 0.59 | 0.60
| True Positive |  | 
| False Positive |  | 
| False Negative |  | 
| Precision | 0. | 0.
| Recall | 0. | 0.
| F1-Score | 0. | 0.


## Ablation Study


In [32]:
ablation_dict = functions.ablation_study_p_dev(
    preds,
    clean_subjects_videos_ground_truth_labels,
    clean_videos_images,
    clean_subjects,
    clean_subjects_videos_code,
    k,
)

 p | TP | FP | FN | Precision | Recall | F1-Score
0.01 | 118 | 810 | 213 | 0.1272 | 0.3565 | 0.1875 |
0.02 | 116 | 794 | 215 | 0.1275 | 0.3505 | 0.1869 |
0.03 | 115 | 775 | 216 | 0.1292 | 0.3474 | 0.1884 |
0.04 | 113 | 764 | 218 | 0.1288 | 0.3414 | 0.1871 |
0.05 | 113 | 749 | 218 | 0.1311 | 0.3414 | 0.1894 |
0.06 | 111 | 737 | 220 | 0.1309 | 0.3353 | 0.1883 |
0.07 | 110 | 719 | 221 | 0.1327 | 0.3323 | 0.1897 |
0.08 | 108 | 704 | 223 | 0.1330 | 0.3263 | 0.1890 |
0.09 | 108 | 693 | 223 | 0.1348 | 0.3263 | 0.1908 |
0.10 | 107 | 676 | 224 | 0.1367 | 0.3233 | 0.1921 |
0.11 | 105 | 662 | 226 | 0.1369 | 0.3172 | 0.1913 |
0.12 | 103 | 649 | 228 | 0.1370 | 0.3112 | 0.1902 |
0.13 | 101 | 634 | 230 | 0.1374 | 0.3051 | 0.1895 |
0.14 | 100 | 621 | 231 | 0.1387 | 0.3021 | 0.1901 |
0.15 | 99 | 607 | 232 | 0.1402 | 0.2991 | 0.1909 |
0.16 | 99 | 599 | 232 | 0.1418 | 0.2991 | 0.1924 |
0.17 | 98 | 584 | 233 | 0.1437 | 0.2961 | 0.1935 |
0.18 | 97 | 565 | 234 | 0.1465 | 0.2931 | 0.1954 |
0.19 | 97 | 553 | 